## **0. Tải bộ dữ liệu**
**Lưu ý:** Nếu bạn không thể sử dụng lệnh gdown để tải bộ dữ liệu vì bị giới hạn số lượt tải, hãy tải bộ dữ liệu thử công và upload lên google drive của mình. Sau đó, sử dụng lệnh dưới đây để copy file dữ liệu vào colab:
```python
from google.colab import drive

drive.mount('/content/drive')
!cp /path/to/dataset/on/your/drive .
```

In [1]:
# https://drive.google.com/file/d/1N7rk-kfnDFIGMeX0ROVTjKh71gcgx-7R/view?usp=sharing
!gdown 1N7rk-kfnDFIGMeX0ROVTjKh71gcgx-7R

Downloading...
From: https://drive.google.com/uc?id=1N7rk-kfnDFIGMeX0ROVTjKh71gcgx-7R
To: /content/2cls_spam_text_cls.csv
100% 486k/486k [00:00<00:00, 12.3MB/s]


## **1. Import các thư viện cần thiết**

In [2]:
import string
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


## **2. Đọc bộ dữ liệu**

In [3]:
DATASET_PATH = '/content/2cls_spam_text_cls.csv'
df = pd.read_csv(DATASET_PATH)
df

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [4]:
messages = df['Message'].values.tolist()
labels = df['Category'].values.tolist()

In [5]:
messages[0]

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [6]:
labels[0]

'ham'

## **3. Chuẩn bị bộ dữ liệu**

### **3.1. Xử lý dữ liệu nhãn**

In [7]:
set(labels)

{'ham', 'spam'}

In [ ]:
label_dict = {
    0: 'ham',
    1: 'spam'
}

In [8]:
le = LabelEncoder()
y = le.fit_transform(labels)
print(f'Classes: {le.classes_}')
print(f'Encoded labels: {y}')

Classes: ['ham' 'spam']
Encoded labels: [0 0 1 ... 0 0 0]


### **3.2. Xử lý dữ liệu đặc trưng**

In [9]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [11]:
def lowercase(text):
    return text.lower()

def punctuation_removal(text):
    translator = text.maketrans('', '', string.punctuation)
    return text.translate(translator)
    # for ch in string.punctuation:
    #     text = text.replace(ch, '')
    # return text

def tokenize(text):
    return text.split() # nltk.word_tokenize(text)

def remove_stopwords(tokens):
    stop_words = nltk.corpus.stopwords.words('english')

    return [token for token in tokens if token not in stop_words]

def stemming(tokens):
    stemmer = nltk.PorterStemmer()

    return [stemmer.stem(token) for token in tokens]

def preprocess_text(text):
    # Your code here
    text = lowercase(text)
    text = punctuation_removal(text)
    tokens = tokenize(text)
    tokens = remove_stopwords(tokens)
    tokens = stemming(tokens)

    return tokens

In [12]:
messages = [preprocess_text(message) for message in messages]

In [14]:
def create_dictionary(messages):
    dictionary = []

    for tokens in messages:
        for token in tokens:
            if token not in dictionary:
                dictionary.append(token)

    return dictionary

dictionary = create_dictionary(messages)

In [16]:
len(dictionary)

8190

In [17]:
def create_features(tokens, dictionary):
    features = np.zeros(len(dictionary))

    for token in tokens:
        if token in dictionary:
            features[dictionary.index(token)] += 1

    return features

In [18]:
X = np.array([create_features(tokens, dictionary) for tokens in messages])

In [21]:
dictionary[:5]

['go', 'jurong', 'point', 'crazi', 'avail']

In [22]:
X.shape

(5572, 8190)

### **3.3. Chia dữ liệu train/val/test**

In [24]:
0.8*5572

4457.6

In [29]:
TEST_SIZE = 0.2 #80%: train 20% val
# TEST_SIZE = 0.125 # 0.875% train: 0.125% test
SEED = 0

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                  test_size=TEST_SIZE,
                                                  shuffle=True,
                                                  random_state=SEED)
# X_train, X_test, y_train, y_test = train_test_split(X_train, y_train,
#                                                     test_size=TEST_SIZE,
#                                                     shuffle=True,
#                                                     random_state=SEED)

In [30]:
X_train.shape

(4457, 8190)

In [28]:
y_train

array([0, 0, 0, ..., 1, 0, 0])

## **4. Huấn luyện mô hình**

In [32]:
%%time
model = MultinomialNB() # GaussianNB()
print('Start training...')
model = model.fit(X_train, y_train)
print('Training completed!')

Start training...
Training completed!
CPU times: user 227 ms, sys: 3.7 ms, total: 231 ms
Wall time: 154 ms


## **5. Đánh giá mô hình**

In [44]:
X_test.shape

(1115, 8190)

In [33]:
# y_val_pred = model.predict(X_val)
y_test_pred = model.predict(X_test)
# val_accuracy = accuracy_score(y_val, y_val_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
# print(f'Val accuracy: {val_accuracy}')
print(f'Test accuracy: {test_accuracy}')

Test accuracy: 0.9820627802690582


In [36]:
len(y_test)

1115

In [37]:
np.sum(y_test == y_test_pred) / len(y_test)

np.float64(0.9820627802690582)

## **6. Thực hiện dự đoán**

In [42]:
def predict(text, model, dictionary):
    processed_text = preprocess_text(text)
    features = create_features(text, dictionary)
    features = np.array(features).reshape(1, -1) # 1x8190
    print(features.shape)
    prediction = model.predict(features)
    print(prediction)
    prediction_cls = le.inverse_transform(prediction)[0]

    return prediction_cls

In [43]:
test_input = 'I am actually thinking a way of doing something useful'
prediction_cls = predict(test_input, model, dictionary)
print(f'Prediction: {prediction_cls}')

(1, 8190)
[0]
Prediction: ham
